# ALL_RSS

### Given that RSS FEEDs update frequently we need to find a way to include the new jobs into the current DB without losing the "old" jobs.

#### Key DataFrames:

######  df = current output of crawler

###### df_a = the previous output of crawler

##### df1_copy = df + df_a (saved to be used by df_a in next iteration)

##### df1 = df + df_a  -> final df 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import mysql.connector
import psycopg2
import numpy as np

In [2]:
df = pd.read_csv('/Users/juanreyesgarcia/Library/CloudStorage/OneDrive-FundacionUniversidaddelasAmericasPuebla/DEVELOPER/PROJECTS/CRAWLER_ALL/yummy_soup_rss.csv')

In [3]:
df_a = pd.read_csv('/Users/juanreyesgarcia/Library/CloudStorage/OneDrive-FundacionUniversidaddelasAmericasPuebla/DEVELOPER/PROJECTS/CRAWLER_ALL/RSS_CRAWLERS/yummy_soup_rss_copy.csv')

##### Play with the settings...

In [4]:
pd.set_option('display.max_colwidth', 150)

In [5]:
pd.set_option("display.max_rows", None)

# EDA Checklist 

In [6]:
df.head()

,title,link,description,pubDate,location
0,Senior Clojure Developer Codete,https://ClojureJobboard.com/clojure-job/ib-remote-senior-clojure-developer-krakow-codete-remotework.html,Remote Codete is not just a software company its a place where techenthusiasts can grow by doing what they love and feel valued for what they are...,NaN,NaN
1,Senior Software Engineer CIS Cloud Services Centripetal,https://ClojureJobboard.com/clojure-job/ha-remote-senior-software-engineer-cis-cloud-services-centripetal-remotework.html,Remote Do you want to be at the forefront of intelligencedriven cybersecurity We at Centripetal are innovators of disruptive cybersecurity soluti...,NaN,NaN
2,FullStack Clojure Developer Appear,https://ClojureJobboard.com/clojure-job/ia-remote-europe-full-stack-clojure-developer-appear-remotework.html,Remote At Appear were looking for a FullStack Clojure Developer to join our team Fulltime Oslo OR Remote About the role Were looking for a Cloju...,NaN,NaN
3,Software Engineer Clojure Backend Comcast,https://ClojureJobboard.com/clojure-job/ic-software-engineer-clojure-backend-philadelphia-comcast.html,Philadelphia United States Comcasts Technology Product Experience organization works at the intersection of media and technology Our innovative ...,NaN,NaN
4,Senior Application Developer Clojure Pennymac,https://ClojureJobboard.com/clojure-job/ii-senior-application-developer-clojure-agoura-hills-pennymac.html,Agoura Hills United States At Pennymac our people are the foundation of our success and at the heart of our dynamic work culture Together we work...,NaN,NaN


In [7]:
df.tail()

,title,link,description,pubDate,location
1379,WordPress Sales Support Specialist,https://jobs.wordpress.net/job/wordpress-sales-support-specialist-5/,Will you be our newest WordPress Sales Support Specialist We are looking for a WordPress Sales Support Specialist to promote highquality WordPress...,Sun 26 Feb 2023 142624 0000,NaN
1380,QA WordPress Plugin Tester,https://jobs.wordpress.net/job/qa-wordpress-plugin-tester-6/,We are looking for a QA 8211 WordPress plugin tester to work with us at WPMU DEV fulltime testing challenging interesting cool new stuff RESPONSIB...,Sun 26 Feb 2023 142619 0000,NaN
1381,WordPress Support Specialist at CampusPress,https://jobs.wordpress.net/job/wordpress-support-specialist-at-campuspress/,We are on the hunt for some epic talent to join our CampusPress family Youll be working on various WordPress projects across WPMU DEV CampusPress ...,Sun 26 Feb 2023 142606 0000,NaN
1382,WordPress Support Specialist,https://jobs.wordpress.net/job/wordpress-support-specialist-6/,As part of Incsub since 2006 WPMU DEV with its awardwinning WordPress plugins hosting worldbeating support and site management tools has helped mi...,Sun 26 Feb 2023 142604 0000,NaN
1383,CampusPress Sales Support Specialist,https://jobs.wordpress.net/job/campuspress-sales-support-specialist-4/,We are looking for some epic talent to join our CampusPress family At CampusPress were experts in bringing innovative web solutions to schools sch...,Sun 26 Feb 2023 142602 0000,NaN


### Describe the DF (current)

In [8]:
df.describe()

,title,link,description,pubDate,location
count,1384,1384,1149,1208,636
unique,1263,1384,1116,1144,178
top,Senior Software Engineer,https://ClojureJobboard.com/clojure-job/ib-remote-senior-clojure-developer-krakow-codete-remotework.html,United States,20230225T2311040000,Remote
freq,8,1,11,7,88


### Describe DF_a (previous)

In [9]:
df_a.describe()

,title,link,description,pubDate,location
count,1142,1142,1006,662,414
unique,1049,1142,968,33,119
top,Senior Software Engineer,https://cryptojobslist.com/jobs/network-engineer-binance-singapore,United States,2023-01-31,Worldwide
freq,9,1,6,67,101


### Checking for missing values DF (current)

In [10]:
df.isnull().sum()

title            0
link             0
description    235
pubDate        176
location       748
dtype: int64

### Checking for missing values DF_a (previous)

In [11]:
df_a.isnull().sum()

title            0
link             0
description    136
pubDate        480
location       728
dtype: int64

### Fill missing values with "NaN" DF (current)

In [12]:
# Fill missing values with "NaN"
df.fillna("NaN", inplace=True)

In [13]:
df.isnull().sum()

title          0
link           0
description    0
pubDate        0
location       0
dtype: int64

### Fill missing values with "NaN" DF_a (previous)

In [14]:
# Fill missing values with "NaN"
df_a.fillna("NaN", inplace=True)

In [15]:
df_a.isnull().sum()

title          0
link           0
description    0
pubDate        0
location       0
dtype: int64

### Checking for duplicates DF (current)

In [16]:
df.duplicated().sum()

0

In [17]:
df.describe()

,title,link,description,pubDate,location
count,1384,1384,1384,1384,1384
unique,1263,1384,1117,1145,179
top,Senior Software Engineer,https://ClojureJobboard.com/clojure-job/ib-remote-senior-clojure-developer-krakow-codete-remotework.html,NaN,NaN,NaN
freq,8,1,235,176,748


### Checking for duplicates DF_a (previous)

In [18]:
df_a.duplicated().sum()

0

# From string to date time...

In [19]:
df_a.loc[:, 'pubDate']  = pd.to_datetime(df_a['pubDate'], errors="coerce", format="%a %d %b %Y", exact=False)
df.loc[:, 'pubDate']  = pd.to_datetime(df['pubDate'], errors="coerce", format="%a %d %b %Y", exact=False)

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_1672/1459423331.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_a.loc[:, 'pubDate']  = pd.to_datetime(df_a['pubDate'], errors="coerce", format="%a %d %b %Y", exact=False)
/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_1672/1459423331.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'pubDate']  = pd.to_datetime(df['pubDate'], errors="coerce", format="%a %d %b %Y", exact=False)


## Filter rows by a date range (this reduces the number of rows... duh)

In [20]:
# filter rows by a date range
start_date = pd.to_datetime('2016-01-01')
end_date = pd.to_datetime('2023-01-24')

#for df_a
date_range_filter_a = (df_a['pubDate'] >= start_date) & (df_a['pubDate'] <= end_date)
df_a = df_a.loc[~date_range_filter_a]

#for df
date_range_filter = (df['pubDate'] >= start_date) & (df['pubDate'] <= end_date)
df = df.loc[~date_range_filter]

In [21]:
df_a = df_a.sort_values(by='pubDate')
df = df.sort_values(by='pubDate')

In [22]:
df.describe(datetime_is_numeric=True)

,pubDate
count,711
mean,2023-02-14 22:12:39.493670912
min,2023-01-25 00:00:00
25%,2023-02-04 00:00:00
50%,2023-02-16 00:00:00
75%,2023-02-27 00:00:00
max,2023-03-02 00:00:00


In [23]:
df_a.describe(datetime_is_numeric=True)

,pubDate
count,0
mean,NaT
min,NaT
25%,NaT
50%,NaT
75%,NaT
max,NaT


In [24]:
df

,title,link,description,pubDate,location
595,Finance Manager Corporate FPA,https://cryptojobslist.com/jobs/finance-manager-corporate-fp-a-binance-singapore,Tags Cryptocurrency Jobs Blockchain Marketing Jobs Blockchain Trading Jobs Web3 Developer Jobs Web3 Engineering Jobs Web3 Exchange Jobs Web...,2023-01-25,Singapore
609,Treasury Analyst,https://cryptojobslist.com/jobs/treasury-analyst-binance-singapore,Tags Cryptocurrency Jobs Cryptocurrency Trading Jobs Blockchain Analyst Jobs Cryptocurrency Exchange Jobs Cryptocurrency Finance Jobs Crypto...,2023-01-25,Singapore
608,Legal Counsel IP Commercial,https://cryptojobslist.com/jobs/legal-counsel-ip-commercial-binance-singapore,Tags Cryptocurrency Jobs Blockchain Trading Jobs Cryptocurrency Legal Jobs Cryptocurrency Exchange Jobs Cryptocurrency Compliance Jobs Crypt...,2023-01-25,Singapore
607,Senior to Mid level Data Backend Engineer Central Data Service Team,https://cryptojobslist.com/jobs/senior-to-mid-level-data-backend-engineer-central-data-service-team-binance-singapore,Tags Cryptocurrency Jobs Blockchain Trading Jobs Blockchain Developer Jobs Web3 Engineering Jobs Cryptocurrency Exchange Jobs Blockchain Sen...,2023-01-25,Singapore
605,Wallet Operations Deputy,https://cryptojobslist.com/jobs/wallet-operations-deputy-binance-singapore,Tags Cryptocurrency Jobs Blockchain Trading Jobs Blockchain Operations Jobs Blockchain Exchange Jobs Cryptocurrency Finance Jobs Cryptocurre...,2023-01-25,Singapore
604,Operations Director Custody,https://cryptojobslist.com/jobs/operations-director-custody-binance-singapore,Tags Cryptocurrency Jobs Cryptocurrency Trading Jobs Blockchain Operations Jobs Web3 Security Jobs Cryptocurrency Business Development Jobs ...,2023-01-25,Singapore
603,Procurement Manager Professional Services,https://cryptojobslist.com/jobs/procurement-manager-professional-services-binance-singapore,Tags Cryptocurrency Jobs Blockchain Marketing Jobs Cryptocurrency Trading Jobs Web3 Legal Jobs Web3 Security Jobs Web3 Exchange Jobs Blockc...,2023-01-25,Singapore
602,Clearing Specialist,https://cryptojobslist.com/jobs/clearing-specialist-binance-singapore,Tags Cryptocurrency Jobs Blockchain Trading Jobs Web3 Exchange Jobs Blockchain Finance Jobs Web3 Data Jobs Blockchain Accounting Jobs Block...,2023-01-25,Singapore
601,Senior Product Manager Fund Management,https://cryptojobslist.com/jobs/senior-product-manager-fund-management-binance-singapore,Tags Cryptocurrency Jobs Blockchain Remote Jobs Blockchain Trading Jobs Cryptocurrency Product Manager Jobs Cryptocurrency Exchange Jobs Web...,2023-01-25,Singapore
600,Data Analyst Lead,https://cryptojobslist.com/jobs/data-analyst-lead-binance-singapore,Tags Cryptocurrency Jobs Blockchain Trading Jobs Web3 Analyst Jobs Web3 Developer Jobs Blockchain Engineering Jobs Cryptocurrency Exchange J...,2023-01-25,Singapore


In [25]:
df.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_1672/3627053830.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe()


,title,link,description,pubDate,location
count,954,954,954,711,954
unique,879,954,788,36,128
top,Senior Software Engineer,https://cryptojobslist.com/jobs/finance-manager-corporate-fp-a-binance-singapore,NaN,2023-03-02 00:00:00,NaN
freq,6,1,139,94,581
first,NaN,NaN,NaN,2023-01-25 00:00:00,NaN
last,NaN,NaN,NaN,2023-03-02 00:00:00,NaN


In [26]:
df.dtypes

title                  object
link                   object
description            object
pubDate        datetime64[ns]
location               object
dtype: object

In [27]:
df['title'] = df['title'].astype(str)
df['link'] = df['link'].astype(str)
df['description'] = df['description'].astype(str)
df['location'] = df['location'].astype(str)

In [28]:
df.dtypes

title                  object
link                   object
description            object
pubDate        datetime64[ns]
location               object
dtype: object

###### Remember 'object' is the pandas data type that is used to represent string data.

In [29]:
df1 = df.copy()

In [30]:
df['description'] = df['description'].str.slice(0, 1000)

In [31]:
# replace NaT values in the DataFrame with None
df = df.replace({np.nan: None, pd.NaT: None})

In [32]:
df

,title,link,description,pubDate,location
595,Finance Manager Corporate FPA,https://cryptojobslist.com/jobs/finance-manager-corporate-fp-a-binance-singapore,Tags Cryptocurrency Jobs Blockchain Marketing Jobs Blockchain Trading Jobs Web3 Developer Jobs Web3 Engineering Jobs Web3 Exchange Jobs Web...,2023-01-25 00:00:00,Singapore
609,Treasury Analyst,https://cryptojobslist.com/jobs/treasury-analyst-binance-singapore,Tags Cryptocurrency Jobs Cryptocurrency Trading Jobs Blockchain Analyst Jobs Cryptocurrency Exchange Jobs Cryptocurrency Finance Jobs Crypto...,2023-01-25 00:00:00,Singapore
608,Legal Counsel IP Commercial,https://cryptojobslist.com/jobs/legal-counsel-ip-commercial-binance-singapore,Tags Cryptocurrency Jobs Blockchain Trading Jobs Cryptocurrency Legal Jobs Cryptocurrency Exchange Jobs Cryptocurrency Compliance Jobs Crypt...,2023-01-25 00:00:00,Singapore
607,Senior to Mid level Data Backend Engineer Central Data Service Team,https://cryptojobslist.com/jobs/senior-to-mid-level-data-backend-engineer-central-data-service-team-binance-singapore,Tags Cryptocurrency Jobs Blockchain Trading Jobs Blockchain Developer Jobs Web3 Engineering Jobs Cryptocurrency Exchange Jobs Blockchain Sen...,2023-01-25 00:00:00,Singapore
605,Wallet Operations Deputy,https://cryptojobslist.com/jobs/wallet-operations-deputy-binance-singapore,Tags Cryptocurrency Jobs Blockchain Trading Jobs Blockchain Operations Jobs Blockchain Exchange Jobs Cryptocurrency Finance Jobs Cryptocurre...,2023-01-25 00:00:00,Singapore
604,Operations Director Custody,https://cryptojobslist.com/jobs/operations-director-custody-binance-singapore,Tags Cryptocurrency Jobs Cryptocurrency Trading Jobs Blockchain Operations Jobs Web3 Security Jobs Cryptocurrency Business Development Jobs ...,2023-01-25 00:00:00,Singapore
603,Procurement Manager Professional Services,https://cryptojobslist.com/jobs/procurement-manager-professional-services-binance-singapore,Tags Cryptocurrency Jobs Blockchain Marketing Jobs Cryptocurrency Trading Jobs Web3 Legal Jobs Web3 Security Jobs Web3 Exchange Jobs Blockc...,2023-01-25 00:00:00,Singapore
602,Clearing Specialist,https://cryptojobslist.com/jobs/clearing-specialist-binance-singapore,Tags Cryptocurrency Jobs Blockchain Trading Jobs Web3 Exchange Jobs Blockchain Finance Jobs Web3 Data Jobs Blockchain Accounting Jobs Block...,2023-01-25 00:00:00,Singapore
601,Senior Product Manager Fund Management,https://cryptojobslist.com/jobs/senior-product-manager-fund-management-binance-singapore,Tags Cryptocurrency Jobs Blockchain Remote Jobs Blockchain Trading Jobs Cryptocurrency Product Manager Jobs Cryptocurrency Exchange Jobs Web...,2023-01-25 00:00:00,Singapore
600,Data Analyst Lead,https://cryptojobslist.com/jobs/data-analyst-lead-binance-singapore,Tags Cryptocurrency Jobs Blockchain Trading Jobs Web3 Analyst Jobs Web3 Developer Jobs Blockchain Engineering Jobs Cryptocurrency Exchange J...,2023-01-25 00:00:00,Singapore


# PostgreSQL

### This code creates a new table per iteration

##### YOU NEED TO MODIFY THIS SO IT INCLUDES THE ROWS THAT HAVE "NaT"

In [33]:
# create a connection to the PostgreSQL database
cnx = psycopg2.connect(user='postgres', password='3312', host='localhost', port='5432', database='postgres')

# create a cursor object
cursor = cnx.cursor()

# get the name of the next table to create
get_table_name_query = '''
    SELECT COUNT(*) FROM information_schema.tables
    WHERE table_name LIKE 'mytable_%'
'''
cursor.execute(get_table_name_query)
result = cursor.fetchone()
next_table_number = result[0] + 1
next_table_name = 'mytable_{}'.format(next_table_number)

# prepare the SQL query to create a new table
create_table_query = '''
    CREATE TABLE {} (
        title VARCHAR(255),
        link VARCHAR(255),
        description VARCHAR(1000),
        pubDate TIMESTAMP,
        location VARCHAR(255)
    )
'''.format(next_table_name)

# execute the create table query
cursor.execute(create_table_query)

# insert the DataFrame into the PostgreSQL database as a new table
for index, row in df.iterrows():
    insert_query = '''
        INSERT INTO {} (title, link, description, pubDate, location)
        VALUES (%s, %s, %s, %s, %s)
    '''.format(next_table_name)
    values = (row['title'], row['link'], row['description'], row['pubDate'], row['location'])
    cursor.execute(insert_query, values)

# commit the changes to the database
cnx.commit()

# close the cursor and connection
cursor.close()
cnx.close()

# MERGE 

In [34]:
#merged = pd.merge(df, df_a, on=['title','link','description','pubDate','location'], how='outer')

In [35]:
#merged.describe()

#### Check for duplicatesss...

In [36]:
#merged.duplicated(subset='link').sum()

#### Drop them...

In [37]:
#df1 = merged.drop_duplicates(subset=['link'])

In [38]:
#df1.duplicated(subset='link').sum()

In [39]:
#df1.describe()

# Save a copy of the df back into yummy_soup_rss_copy.csv so in the next iterations you will merge the output of the crawler with this df (basically making a loop)

## By doing this we will be able to merge the df that comes from the crawler and the merged df. If this was not done (& we just call the yummy_soup_rss_copy.csv file we would only merge those rows with the new ones, without including the data that was already merged)

In [40]:
#df1_copy = df1.copy()
#df1_copy.to_csv('yummy_soup_rss_copy.csv', index=False)

### Explore categorical data

For categorical data, check the number of unique values in each column 

In [41]:
#df1.nunique()

In [42]:
# Visualise the distribution of a categorical column using a bar chart
#df1['pubDate'].value_counts().plot(kind='bar', figsize=(14, 8))
#plt.show()

### Sort the dataframe by the date column in ascending order

In [43]:
#df1 = df1.sort_values(by='pubDate', ascending=False)

### Reset index...

In [44]:
#df1.reset_index(drop=True)

# GIVE ME THOSE JOBSSSS

## Only WorldWide - 2023

#### Given that some dates are NaN it is impossible to create a perfectly tailored df of solely worldwide jobs

##### I have tried to narrow down the search but there is still some jobs that are not fully worldwide. Although, the majority is indeed from anywhere in the world.

In [45]:
key_words = ['worldwide', 'international', 'anywhere']
regex = '|'.join(key_words)


worldwide = df1[df1['title'].str.contains(regex, case=False) | df1['description'].str.contains(regex, case=False) | df1['location'].str.contains(regex, case=False)]

In [46]:
worldwide.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_1672/3825536958.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  worldwide.describe()


,title,link,description,pubDate,location
count,214,214,214,154,214
unique,208,214,212,29,35
top,Senior Backend Engineer,https://cryptojobslist.com/jobs/finance-manager-corporate-fp-a-binance-singapore,Remote Gcore in the nutshell We are an international leader in public cloud and edge computing content delivery hosting and security solutions We...,2023-01-25 00:00:00,NaN
freq,2,1,2,29,83
first,NaN,NaN,NaN,2023-01-25 00:00:00,NaN
last,NaN,NaN,NaN,2023-03-02 00:00:00,NaN


In [47]:
worldwide

,title,link,description,pubDate,location
595,Finance Manager Corporate FPA,https://cryptojobslist.com/jobs/finance-manager-corporate-fp-a-binance-singapore,Tags Cryptocurrency Jobs Blockchain Marketing Jobs Blockchain Trading Jobs Web3 Developer Jobs Web3 Engineering Jobs Web3 Exchange Jobs Web...,2023-01-25,Singapore
609,Treasury Analyst,https://cryptojobslist.com/jobs/treasury-analyst-binance-singapore,Tags Cryptocurrency Jobs Cryptocurrency Trading Jobs Blockchain Analyst Jobs Cryptocurrency Exchange Jobs Cryptocurrency Finance Jobs Crypto...,2023-01-25,Singapore
608,Legal Counsel IP Commercial,https://cryptojobslist.com/jobs/legal-counsel-ip-commercial-binance-singapore,Tags Cryptocurrency Jobs Blockchain Trading Jobs Cryptocurrency Legal Jobs Cryptocurrency Exchange Jobs Cryptocurrency Compliance Jobs Crypt...,2023-01-25,Singapore
607,Senior to Mid level Data Backend Engineer Central Data Service Team,https://cryptojobslist.com/jobs/senior-to-mid-level-data-backend-engineer-central-data-service-team-binance-singapore,Tags Cryptocurrency Jobs Blockchain Trading Jobs Blockchain Developer Jobs Web3 Engineering Jobs Cryptocurrency Exchange Jobs Blockchain Sen...,2023-01-25,Singapore
605,Wallet Operations Deputy,https://cryptojobslist.com/jobs/wallet-operations-deputy-binance-singapore,Tags Cryptocurrency Jobs Blockchain Trading Jobs Blockchain Operations Jobs Blockchain Exchange Jobs Cryptocurrency Finance Jobs Cryptocurre...,2023-01-25,Singapore
604,Operations Director Custody,https://cryptojobslist.com/jobs/operations-director-custody-binance-singapore,Tags Cryptocurrency Jobs Cryptocurrency Trading Jobs Blockchain Operations Jobs Web3 Security Jobs Cryptocurrency Business Development Jobs ...,2023-01-25,Singapore
603,Procurement Manager Professional Services,https://cryptojobslist.com/jobs/procurement-manager-professional-services-binance-singapore,Tags Cryptocurrency Jobs Blockchain Marketing Jobs Cryptocurrency Trading Jobs Web3 Legal Jobs Web3 Security Jobs Web3 Exchange Jobs Blockc...,2023-01-25,Singapore
602,Clearing Specialist,https://cryptojobslist.com/jobs/clearing-specialist-binance-singapore,Tags Cryptocurrency Jobs Blockchain Trading Jobs Web3 Exchange Jobs Blockchain Finance Jobs Web3 Data Jobs Blockchain Accounting Jobs Block...,2023-01-25,Singapore
601,Senior Product Manager Fund Management,https://cryptojobslist.com/jobs/senior-product-manager-fund-management-binance-singapore,Tags Cryptocurrency Jobs Blockchain Remote Jobs Blockchain Trading Jobs Cryptocurrency Product Manager Jobs Cryptocurrency Exchange Jobs Web...,2023-01-25,Singapore
600,Data Analyst Lead,https://cryptojobslist.com/jobs/data-analyst-lead-binance-singapore,Tags Cryptocurrency Jobs Blockchain Trading Jobs Web3 Analyst Jobs Web3 Developer Jobs Blockchain Engineering Jobs Cryptocurrency Exchange J...,2023-01-25,Singapore


## Analyst - WorldWide

In [48]:

analyst = worldwide[worldwide['title'].str.contains('analyst', case=False) | worldwide['description'].str.contains('analyst', case=False)]


In [49]:
analyst.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_1672/1449045089.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  analyst.describe()


,title,link,description,pubDate,location
count,10,10,10,7,10
unique,10,10,10,5,7
top,Treasury Analyst,https://cryptojobslist.com/jobs/treasury-analyst-binance-singapore,Tags Cryptocurrency Jobs Cryptocurrency Trading Jobs Blockchain Analyst Jobs Cryptocurrency Exchange Jobs Cryptocurrency Finance Jobs Crypto...,2023-01-25 00:00:00,Singapore
freq,1,1,1,2,4
first,NaN,NaN,NaN,2023-01-25 00:00:00,NaN
last,NaN,NaN,NaN,2023-02-23 00:00:00,NaN


In [50]:
analyst

,title,link,description,pubDate,location
609,Treasury Analyst,https://cryptojobslist.com/jobs/treasury-analyst-binance-singapore,Tags Cryptocurrency Jobs Cryptocurrency Trading Jobs Blockchain Analyst Jobs Cryptocurrency Exchange Jobs Cryptocurrency Finance Jobs Crypto...,2023-01-25,Singapore
600,Data Analyst Lead,https://cryptojobslist.com/jobs/data-analyst-lead-binance-singapore,Tags Cryptocurrency Jobs Blockchain Trading Jobs Web3 Analyst Jobs Web3 Developer Jobs Blockchain Engineering Jobs Cryptocurrency Exchange J...,2023-01-25,Singapore
488,General Application,https://cryptojobslist.com/jobs/general-application-treehouse-singapore,Tags Cryptocurrency Jobs Cryptocurrency Marketing Jobs Blockchain Research Jobs Web3 Internship Jobs Blockchain Analyst Jobs Cryptocurrency ...,2023-01-31,Singapore
549,Java Development Engineer SecurityBinance Accelerator Programme,https://cryptojobslist.com/jobs/java-development-engineer-security-binance-accelerator-programme-binance-asia,Tags Cryptocurrency Jobs Cryptocurrency Trading Jobs Blockchain Security Jobs Cryptocurrency Developer Jobs Web3 Exchange Jobs Cryptocurrenc...,2023-01-31,Asia
357,Cybercrimes Investigator,https://cryptojobslist.com/jobs/cybercrimes-investigator-chainalysis-washington-d-c-remote,Tags Cryptocurrency Jobs Cryptocurrency Remote Jobs Cryptocurrency Research Jobs Cryptocurrency Sales Jobs Cryptocurrency Engineering Jobs W...,2023-02-06,Washington DC Remote
241,Accounts Payable Analyst,https://cryptojobslist.com/jobs/accounts-payable-analyst-parity-technologies-london-england-united-kingdom,Tags Cryptocurrency Jobs Web3 Analyst Jobs Cryptocurrency Operations Jobs Cryptocurrency Finance Jobs Web3 Accounting Jobs Web3 Full Time Jo...,2023-02-17,London England United Kingdom
145,Binance Accelerator Programme Data analyst intern,https://cryptojobslist.com/jobs/binance-accelerator-programme-data-analyst-intern-binance-singapore,Tags Cryptocurrency Jobs Cryptocurrency Trading Jobs Blockchain Internship Jobs Blockchain Analyst Jobs Blockchain Engineering Jobs Web3 Exc...,2023-02-23,Singapore
1152,Technical Analyst,https://remoteOK.com/jobs/201886,1kx is an earlystage fund that helps founders launch and bootstrap token networks We primarily lead Seed and Series A investment rounds with a foc...,NaT,Remote Worldwide
1186,Validator Research Analyst,https://remoteOK.com/jobs/200374,As stewards of the Polkadot and Substrate ecosystem Parity is laying the foundation for a better web which respects the freedom and data of indivi...,NaT,Worldwide
1228,Remote Senior eCommerce Inventory and Demand Planning Associate,https://www.virtualvocations.com/job/remote-senior-ecommerce-inventory-and-demand-planning-associate-2016517-i.html,SR ECOMMERCE INVENTORY DEMAND PLANNING ASSOCIATE There are NO Limits is our motto We are a fast paced high growth company We seek people who have...,NaT,NaN


## Sales - Worldwide

In [51]:
sales = worldwide[worldwide['title'].str.contains('sales', case=False) | worldwide['description'].str.contains('sales', case=False)]

In [52]:
sales.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_1672/3195154911.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  sales.describe()


,title,link,description,pubDate,location
count,20,20,20,14,20
unique,20,20,20,9,6
top,Technical Product Lead Sygma,https://cryptojobslist.com/jobs/technical-product-lead-sygma-chainsafe-systems-remote,Tags Cryptocurrency Jobs Web3 Remote Jobs Blockchain Marketing Jobs Cryptocurrency Research Jobs Cryptocurrency Sales Jobs Blockchain Commun...,2023-03-01 00:00:00,NaN
freq,1,1,1,4,12
first,NaN,NaN,NaN,2023-02-02 00:00:00,NaN
last,NaN,NaN,NaN,2023-03-01 00:00:00,NaN


In [53]:
sales

,title,link,description,pubDate,location
423,Technical Product Lead Sygma,https://cryptojobslist.com/jobs/technical-product-lead-sygma-chainsafe-systems-remote,Tags Cryptocurrency Jobs Web3 Remote Jobs Blockchain Marketing Jobs Cryptocurrency Research Jobs Cryptocurrency Sales Jobs Blockchain Commun...,2023-02-02,Remote
378,Customer Marketing Manager,https://cryptojobslist.com/jobs/customer-marketing-manager-chainalysis-remote-usa,Tags Cryptocurrency Jobs Blockchain Remote Jobs Cryptocurrency Marketing Jobs Cryptocurrency Sales Jobs Web3 Content Jobs Blockchain Operati...,2023-02-04,Remote USA
357,Cybercrimes Investigator,https://cryptojobslist.com/jobs/cybercrimes-investigator-chainalysis-washington-d-c-remote,Tags Cryptocurrency Jobs Cryptocurrency Remote Jobs Cryptocurrency Research Jobs Cryptocurrency Sales Jobs Cryptocurrency Engineering Jobs W...,2023-02-06,Washington DC Remote
75,Director of Health Plan Strategy Sales New 911 Telehealth Technology at MD Ally Technologies Inc Anywhere,https://www.crunchboard.com/jobs/193311778-director-of-health-plan-strategy-sales-new-911-telehealth-technology-at-md-ally-technologies-inc,ABOUT MD ALLY Each year millions of nonemergency calls are made to 911 that decrease ambulance availability overcrowd EDs and drive billions in,2023-02-06,NaN
340,Web designer developer frontend backend with Graphic design skills Writer,https://cryptojobslist.com/jobs/web-designer-developer-frontend-backed-with-graphic-design-skills-writer-gd10-capital-worldwide,Tags Cryptocurrency Jobs Web3 Marketing Jobs Blockchain Sales Jobs Web3 Content Jobs Web3 Developer Jobs Web3 Ui Ux Jobs Web3 Designer Jobs...,2023-02-08,Worldwide
313,Sales Representative,https://cryptojobslist.com/jobs/sales-representative-aurora-labs-remote-1,Tags Cryptocurrency Jobs Blockchain Remote Jobs Blockchain Sales Jobs Web3 Business Development Jobs Web3 Full Time Jobs Blockchain Evm Jobs...,2023-02-10,Remote
1341,Aha Senior Product Marketing Manager,https://weworkremotely.com/remote-jobs/aha-senior-product-marketing-manager,Headquarters United States URL httpwwwahaio Aha helps companies build lovable software We provide the worlds 1 product development tools Aha Ro...,2023-02-16,NaN
1314,Salesflowio Account Executive Canada or USA only,https://weworkremotely.com/remote-jobs/salesflow-io-account-executive-canada-or-usa-only,Headquarters London URL httpssalesflowio Salesflow is a B2B growth hacking amp AI sales automation tool that allows companies to generate new B...,2023-02-27,NaN
1310,Hygraph Sales Development Representative fmd Remote within the Eastern Time Zone,https://weworkremotely.com/remote-jobs/hygraph-sales-development-representative-f-m-d-remote-within-the-eastern-time-zone,Headquarters Dircksenstrae 47 10178 Berlin Germany URL httpshygraphcom How you will make an impactAs our Sales Development Representative SDR ...,2023-02-28,NaN
1307,GovAssist LLC Ad Operations Specialist,https://weworkremotely.com/remote-jobs/govassist-llc-ad-operations-specialist,Headquarters US URL httpsgovassistbamboohrcomjobs We are GovAssistcom GovAssist LLC is the leading company specialized in immigration consulta...,2023-02-28,NaN


## Python - Worldwide

In [54]:
python = worldwide[worldwide['title'].str.contains('python', case=False) | worldwide['description'].str.contains('python', case=False)]

In [55]:
python.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_1672/349435123.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  python.describe()


,title,link,description,pubDate,location
count,23,23,23,16,23
unique,23,23,23,9,6
top,Data Analyst Lead,https://cryptojobslist.com/jobs/data-analyst-lead-binance-singapore,Tags Cryptocurrency Jobs Blockchain Trading Jobs Web3 Analyst Jobs Web3 Developer Jobs Blockchain Engineering Jobs Cryptocurrency Exchange J...,2023-01-25 00:00:00,Singapore
freq,1,1,1,6,8
first,NaN,NaN,NaN,2023-01-25 00:00:00,NaN
last,NaN,NaN,NaN,2023-03-02 00:00:00,NaN


In [56]:
python

,title,link,description,pubDate,location
600,Data Analyst Lead,https://cryptojobslist.com/jobs/data-analyst-lead-binance-singapore,Tags Cryptocurrency Jobs Blockchain Trading Jobs Web3 Analyst Jobs Web3 Developer Jobs Blockchain Engineering Jobs Cryptocurrency Exchange J...,2023-01-25,Singapore
596,Binance Accelerator Programme Quality Assurance Engineer Big Data,https://cryptojobslist.com/jobs/binance-accelerator-programme-quality-assurance-engineer-big-data-binance-singapore,Tags Cryptocurrency Jobs Cryptocurrency Trading Jobs Blockchain Operations Jobs Blockchain Developer Jobs Web3 Engineering Jobs Blockchain E...,2023-01-25,Singapore
591,Binance Accelerator Programme Security Researcher,https://cryptojobslist.com/jobs/binance-accelerator-programme-security-researcher-binance-singapore,Tags Cryptocurrency Jobs Cryptocurrency Trading Jobs Cryptocurrency Research Jobs Blockchain Security Jobs Blockchain Developer Jobs Blockch...,2023-01-25,Singapore
588,Binance Accelerator Programme Quality Assurance Engineer,https://cryptojobslist.com/jobs/binance-accelerator-programme-quality-assurance-engineer-binance-singapore,Tags Cryptocurrency Jobs Cryptocurrency Trading Jobs Cryptocurrency Operations Jobs Web3 Developer Jobs Web3 Engineering Jobs Blockchain Exc...,2023-01-25,Singapore
581,Senior Machine Learning Engineer Anomaly Detection,https://cryptojobslist.com/jobs/senior-machine-learning-engineer-anomaly-detection-binance-singapore,Tags Cryptocurrency Jobs Web3 Remote Jobs Web3 Trading Jobs Cryptocurrency Full Time Jobs Blockchain Developer Jobs Web3 Exchange Jobs Web3...,2023-01-25,Singapore
612,Binance Accelerator Programme Data Engineer,https://cryptojobslist.com/jobs/binance-accelerator-programme-data-engineer-binance-singapore,Tags Cryptocurrency Jobs Cryptocurrency Remote Jobs Cryptocurrency Trading Jobs Blockchain Internship Jobs Blockchain Developer Jobs Cryptoc...,2023-01-25,Singapore
130,DeFi Researcher Data Scientist,https://cryptojobslist.com/jobs/defi-researcher-data-scientist-almanak-remote,Tags Cryptocurrency Jobs Web3 Remote Jobs Blockchain Research Jobs Blockchain Data Jobs Web3 Defi Jobs At Almanak we utilize knowledge data a...,2023-01-27,NaN
548,Binance Accelerator Programme Smart Contract Security Engineer,https://cryptojobslist.com/jobs/binance-accelerator-programme-smart-contract-security-engineer-binance-georgia-tbilisi,Tags Cryptocurrency Jobs Web3 Trading Jobs Blockchain Community Jobs Blockchain Security Jobs Blockchain Developer Jobs Cryptocurrency Excha...,2023-01-31,Georgia Tbilisi
462,Senior Software Engineer Lotus,https://cryptojobslist.com/jobs/senior-software-engineer-lotus-protocol-labs-remote,Tags Cryptocurrency Jobs Cryptocurrency Remote Jobs Cryptocurrency Research Jobs Blockchain Community Jobs Blockchain Security Jobs Web3 Dev...,2023-02-02,Remote
453,Binance Accelerator Programme SRE Engineer,https://cryptojobslist.com/jobs/binance-accelerator-programme-sre-engineer-binance-singapore,Tags Cryptocurrency Jobs Blockchain Devops Jobs Cryptocurrency Trading Jobs Cryptocurrency Operations Jobs Web3 Security Jobs Web3 Developer...,2023-02-02,Singapore


## SQL - Worldwide

In [57]:
sql = worldwide[worldwide['title'].str.contains('sql', case=False) | worldwide['description'].str.contains('sql', case=False)]

In [58]:
sql.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_1672/1629035654.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  sql.describe()


,title,link,description,pubDate,location
count,21,21,21,18,21
unique,21,21,21,10,6
top,Finance Manager Corporate FPA,https://cryptojobslist.com/jobs/finance-manager-corporate-fp-a-binance-singapore,Tags Cryptocurrency Jobs Blockchain Marketing Jobs Blockchain Trading Jobs Web3 Developer Jobs Web3 Engineering Jobs Web3 Exchange Jobs Web...,2023-01-25 00:00:00,Singapore
freq,1,1,1,6,7
first,NaN,NaN,NaN,2023-01-25 00:00:00,NaN
last,NaN,NaN,NaN,2023-02-28 00:00:00,NaN


In [59]:
sql

,title,link,description,pubDate,location
595,Finance Manager Corporate FPA,https://cryptojobslist.com/jobs/finance-manager-corporate-fp-a-binance-singapore,Tags Cryptocurrency Jobs Blockchain Marketing Jobs Blockchain Trading Jobs Web3 Developer Jobs Web3 Engineering Jobs Web3 Exchange Jobs Web...,2023-01-25,Singapore
600,Data Analyst Lead,https://cryptojobslist.com/jobs/data-analyst-lead-binance-singapore,Tags Cryptocurrency Jobs Blockchain Trading Jobs Web3 Analyst Jobs Web3 Developer Jobs Blockchain Engineering Jobs Cryptocurrency Exchange J...,2023-01-25,Singapore
599,Java Engineer Financial Product,https://cryptojobslist.com/jobs/java-engineer-financial-product-binance-singapore,Tags Cryptocurrency Jobs Cryptocurrency Trading Jobs Blockchain Developer Jobs Cryptocurrency Engineering Jobs Blockchain Exchange Jobs Web3...,2023-01-25,Singapore
587,Senior Java Engineer Core Exchange System,https://cryptojobslist.com/jobs/senior-java-engineer-core-exchange-system-binance-singapore,Tags Cryptocurrency Jobs Cryptocurrency Remote Jobs Blockchain Trading Jobs Web3 Security Jobs Blockchain Developer Jobs Blockchain Engineer...,2023-01-25,Singapore
582,Manager Commercial Finance,https://cryptojobslist.com/jobs/manager-commercial-finance-binance-singapore,Tags Cryptocurrency Jobs Blockchain Marketing Jobs Blockchain Trading Jobs Web3 Exchange Jobs Web3 Executive Jobs Cryptocurrency Senior Jobs...,2023-01-25,Singapore
612,Binance Accelerator Programme Data Engineer,https://cryptojobslist.com/jobs/binance-accelerator-programme-data-engineer-binance-singapore,Tags Cryptocurrency Jobs Cryptocurrency Remote Jobs Cryptocurrency Trading Jobs Blockchain Internship Jobs Blockchain Developer Jobs Cryptoc...,2023-01-25,Singapore
576,Senior Backend Developer Remote,https://cryptojobslist.com/jobs/senior-backend-developer-remote-dappradar-remote-1,Tags Cryptocurrency Jobs Blockchain Remote Jobs Cryptocurrency Developer Jobs Blockchain Web3 Jobs Web3 Senior Jobs Blockchain Nft Jobs Blo...,2023-01-27,NaN
497,Senior Backend Engineer,https://cryptojobslist.com/jobs/senior-backend-engineer-ethglobal-remote-us-canada,Tags Cryptocurrency Jobs Web3 Remote Jobs Web3 Marketing Jobs Blockchain Community Jobs Blockchain Security Jobs Blockchain Full Time Jobs ...,2023-01-31,Remote US Canada
502,Full Stack Software Engineer,https://cryptojobslist.com/jobs/full-stack-software-engineer-ethglobal-remote-us-canada,Tags Cryptocurrency Jobs Blockchain Remote Jobs Cryptocurrency Marketing Jobs Blockchain Community Jobs Blockchain Full Time Jobs Blockchain...,2023-01-31,Remote US Canada
549,Java Development Engineer SecurityBinance Accelerator Programme,https://cryptojobslist.com/jobs/java-development-engineer-security-binance-accelerator-programme-binance-asia,Tags Cryptocurrency Jobs Cryptocurrency Trading Jobs Blockchain Security Jobs Cryptocurrency Developer Jobs Web3 Exchange Jobs Cryptocurrenc...,2023-01-31,Asia
